# Data

Import libraries:

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import catboost as cb
import gc

Import data & combine transaction and idendity columns:

In [2]:
train_transaction = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/train_transaction.csv')
train_identity = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/train_identity.csv')

train = pd.merge(train_transaction, train_identity, on="TransactionID", how="left")
train = train.set_index("TransactionID", drop="True")
del train_transaction, train_identity
gc.collect()

test_transaction = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/test_transaction.csv')
test_identity = pd.read_csv('/home/fatihakyon/dev/inzva/ieee-fraud-detection/data/test_identity.csv')

test = pd.merge(test_transaction, test_identity, on="TransactionID", how="left")
test = test.set_index("TransactionID", drop="True")
del test_transaction, test_identity
gc.collect()

0

In [3]:
train.shape

(590540, 433)

In [4]:
test.shape

(506691, 432)

In [5]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


Rename test data columns:

In [3]:
mapping = {}
for column_name in test.columns:
    mapping[column_name] = column_name.replace("-", "_")
test.rename(columns=mapping, inplace=True)

In [7]:
test.columns

Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=432)

Reduce memory usage:

In [4]:
import numpy as np


def reduce_mem_usage(df):
    """
    From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

    Iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            pass
            # df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Memory usage of dataframe is 1955.37 MB
Memory usage after optimization is: 648.22 MB
Decreased by 66.8%
Memory usage of dataframe is 1673.87 MB
Memory usage after optimization is: 563.43 MB
Decreased by 66.3%


Split train into train/val:

In [5]:
VAL_SPLIT = 0.2
# Split train into train/val
y = train["isFraud"].copy()
X = train.drop("isFraud", axis=1)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=VAL_SPLIT, random_state=13
)

X_test = test.copy()
del train, test, X, y

X_train = X_train.fillna(-999)
X_val = X_val.fillna(-999)
X_test = X_test.fillna(-999)

Categorical features:

In [6]:
categorical_features = []
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        categorical_features.append(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values) + list(X_val[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_val[f] = lbl.transform(list(X_val[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values)) 

# CatBoost

In [7]:
clf = cb.CatBoostClassifier(n_estimators=200,
                        learning_rate=0.05,
                        metric_period=500,
                        od_wait=500,
                        task_type='CPU',
                        depth=8)

In [10]:
clf.fit(X_train, y_train, cat_features=categorical_features)

0:	learn: 0.6113626	total: 145ms	remaining: 28.8s
199:	learn: 0.0947855	total: 26.2s	remaining: 0us


Results:

In [8]:
def calculate_scores(estimator, X_val, y_val):
    y_val_prediction = estimator.predict(X_val)
    y_val_proba = estimator.predict_proba(X_val)[:, 1]

    conf_matrix = metrics.confusion_matrix(y_val, y_val_prediction)
    accuracy_score = metrics.accuracy_score(y_val, y_val_prediction)
    roc_auc_score = metrics.roc_auc_score(y_val, y_val_proba)
    f1_score = metrics.f1_score(y_val, y_val_prediction)
    classification_report = metrics.classification_report(y_val, y_val_prediction)

    print("Confusion Matrix: \n%s" % str(conf_matrix))
    print("\nAccuracy: %.4f" % accuracy_score)
    print("\nAUC: %.4f" % roc_auc_score)
    print("\nF1 Score: %.4f" % f1_score)
    print("\nClassification Report: \n",classification_report)

    return {
        "conf_matrix": conf_matrix,
        "accuracy_score": accuracy_score,
        "roc_auc_score": roc_auc_score,
        "f1_score": f1_score,
        "classification_report": classification_report
    }

In [16]:
_ = calculate_scores(clf, X_val, y_val)

Confusion Matrix: 
[[113854    118]
 [  2787   1349]]

Accuracy: 0.9754

AUC: 0.8755

F1 Score: 0.4815

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    113972
           1       0.92      0.33      0.48      4136

    accuracy                           0.98    118108
   macro avg       0.95      0.66      0.73    118108
weighted avg       0.97      0.98      0.97    118108



# Hyperarameter Optimization

Randomized Search:

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt  

param_grid = {
    'silent': [False],
    'learning_rate': sp_randFloat(),
    'n_estimators': [200],
    'depth': sp_randInt(6, 16),
    'l2_leaf_reg':[3,1,5,10,100],
    'loss_function': ['Logloss', 'CrossEntropy']
}

def perform_random_search(
    estimator, X_train, X_val, y_train, y_val, param_grid, scoring=None
):
    hyperparam_optimizer = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        scoring=scoring,
        cv=2,
        n_iter=20,
        n_jobs=1,
        refit=True,
        random_state=13,
    )
    hyperparam_optimizer.fit(X_train, y_train, eval_set=(X_val, y_val))

    return hyperparam_optimizer.best_estimator_

In [11]:
# accuracy score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='accuracy')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3729272	test: 0.3732352	best: 0.3732352 (0)	total: 134ms	remaining: 26.6s
199:	learn: 0.0609253	test: 0.0762992	best: 0.0762992 (199)	total: 16s	remaining: 0us

bestTest = 0.07629918512
bestIteration = 199



0:	learn: 0.3717054	test: 0.3720828	best: 0.3720828 (0)	total: 77.2ms	remaining: 15.4s
199:	learn: 0.0618371	test: 0.0758618	best: 0.0758618 (199)	total: 16s	remaining: 0us

bestTest = 0.07586180293
bestIteration = 199



0:	learn: 0.4354961	test: 0.4357128	best: 0.4357128 (0)	total: 74.7ms	remaining: 14.9s
199:	learn: 0.0797422	test: 0.0853030	best: 0.0853030 (199)	total: 15.9s	remaining: 0us

bestTest = 0.0853030195
bestIteration = 199



0:	learn: 0.4342225	test: 0.4344876	best: 0.4344876 (0)	total: 76.7ms	remaining: 15.3s
199:	learn: 0.0840276	test: 0.0885463	best: 0.0885463 (199)	total: 14.4s	remaining: 0us

bestTest = 0.08854631176
bestIteration = 199



0:	learn: 0.1663067	test: 0.1666409	best: 0.1666409 (0)	total: 119ms	remaining: 23.6s
199:	learn: 0.0397854	test: 0.0735952	best: 0.0735952 (199)	total: 23.6s	remaining: 0us

bestTest = 0.07359518541
bestIteration = 199



...

0:	learn: 0.1719746	test: 0.1726591	best: 0.1726591 (0)	total: 687ms	remaining: 2m 16s
199:	learn: 0.0096104	test: 0.0816150	best: 0.0770208 (91)	total: 2m 19s	remaining: 0us

bestTest = 0.07702082961
bestIteration = 91

Shrink model to first 92 iterations.


0:	learn: 0.1722226	test: 0.1734552	best: 0.1734552 (0)	total: 690ms	remaining: 2m 17s
199:	learn: 0.0095227	test: 0.0814562	best: 0.0768382 (78)	total: 2m 20s	remaining: 0us

bestTest = 0.07683818156
bestIteration = 78

Shrink model to first 79 iterations.


0:	learn: 0.1334633	test: 0.1338029	best: 0.1338029 (0)	total: 57.3ms	remaining: 11.4s
199:	learn: 0.0525424	test: 0.0768041	best: 0.0768041 (199)	total: 11.3s	remaining: 0us

bestTest = 0.07680409854
bestIteration = 199



0:	learn: 0.1324851	test: 0.1334653	best: 0.1334653 (0)	total: 54.3ms	remaining: 10.8s
199:	learn: 0.0528575	test: 0.0781396	best: 0.0781357 (196)	total: 11.2s	remaining: 0us

bestTest = 0.07813565154
bestIteration = 196

Shrink model to first 197 iterations.


0:	learn: 0.1734551	test: 0.1746899	best: 0.1746899 (0)	total: 2.78s	remaining: 9m 13s
199:	learn: 0.0159368	test: 0.0550455	best: 0.0550294 (195)	total: 9m 15s	remaining: 0us

bestTest = 0.05502935718
bestIteration = 195

Shrink model to first 196 iterations.
Confusion Matrix: 
[[113824    148]
 [  1559   2577]]

Accuracy: 0.9855

AUC: 0.9570

F1 Score: 0.7512

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    113972
           1       0.95      0.62      0.75      4136

    accuracy                           0.99    118108
   macro avg       0.97      0.81      0.87    118108
weighted avg       0.99      0.99      0.98    118108



In [ ]:
# roc auc score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='roc_auc')
_ = calculate_scores(best_estimator, X_val, y_val)

0:	learn: 0.3729272	test: 0.3732352	best: 0.3732352 (0)	total: 76.1ms	remaining: 15.1s
199:	learn: 0.0609253	test: 0.0762992	best: 0.0762992 (199)	total: 15.2s	remaining: 0us

bestTest = 0.07629918512
bestIteration = 199



0:	learn: 0.3717054	test: 0.3720828	best: 0.3720828 (0)	total: 72.6ms	remaining: 14.4s
199:	learn: 0.0618371	test: 0.0758618	best: 0.0758618 (199)	total: 15.1s	remaining: 0us

bestTest = 0.07586180293
bestIteration = 199



0:	learn: 0.4354961	test: 0.4357128	best: 0.4357128 (0)	total: 72.4ms	remaining: 14.4s
199:	learn: 0.0797422	test: 0.0853030	best: 0.0853030 (199)	total: 14.4s	remaining: 0us

bestTest = 0.0853030195
bestIteration = 199



0:	learn: 0.4342225	test: 0.4344876	best: 0.4344876 (0)	total: 72.3ms	remaining: 14.4s
199:	learn: 0.0840276	test: 0.0885463	best: 0.0885463 (199)	total: 13.6s	remaining: 0us

bestTest = 0.08854631176
bestIteration = 199



0:	learn: 0.1663067	test: 0.1666409	best: 0.1666409 (0)	total: 109ms	remaining: 21.7s
199:	learn: 0.0397854	test: 0.0735952	best: 0.0735952 (199)	total: 22.1s	remaining: 0us

bestTest = 0.07359518541
bestIteration = 199



...

0:	learn: 0.4327882	test: 0.4330441	best: 0.4330441 (0)	total: 2.46s	remaining: 8m 9s
199:	learn: 0.0691884	test: 0.0804477	best: 0.0804477 (199)	total: 8m 25s	remaining: 0us

bestTest = 0.08044769857
bestIteration = 199



0:	learn: 0.1292971	test: 0.1297462	best: 0.1297462 (0)	total: 282ms	remaining: 56.1s
199:	learn: 0.0306598	test: 0.0775629	best: 0.0770253 (170)	total: 58s	remaining: 0us

bestTest = 0.07702531745
bestIteration = 170

Shrink model to first 171 iterations.


0:	learn: 0.1278995	test: 0.1290428	best: 0.1290428 (0)	total: 289ms	remaining: 57.5s
199:	learn: 0.0325193	test: 0.0772116	best: 0.0771474 (189)	total: 59.1s	remaining: 0us

bestTest = 0.07714737491
bestIteration = 189

Shrink model to first 190 iterations.


0:	learn: 0.3194216	test: 0.3199054	best: 0.3199054 (0)	total: 2.42s	remaining: 8m 2s
199:	learn: 0.0078564	test: 0.0705343	best: 0.0690375 (138)	total: 8m 19s	remaining: 0us

bestTest = 0.0690375478
bestIteration = 138

Shrink model to first 139 iterations.


0:	learn: 0.3201304	test: 0.3208861	best: 0.3208861 (0)	total: 2.47s	remaining: 8m 11s


In [ ]:
# f1 score
best_estimator = perform_random_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='f1')
_ = calculate_scores(best_estimator, X_val, y_val)

Bayesian Search:

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

param_grid = {
    'silent': [False],
    'learning_rate': Real(0.01, 0.3),
    'iterations': Integer(10, 100),
    'depth': Integer(6, 20),
    'l2_leaf_reg':[3,1,5,10,100],
    'loss_function': ['Logloss', 'CrossEntropy']
}

def perform_bayes_search(
    estimator, X_train, X_val, y_train, y_val, param_grid, scoring=None
):
    hyperparam_optimizer = BayesSearchCV(
        estimator=estimator,
        search_spaces=param_grid,
        scoring=scoring,
        cv=2,
        n_iter=20,
        n_jobs=1,
        refit=True,
        return_train_score=False,
        optimizer_kwargs={"base_estimator": "GP"},
        random_state=13,
        fit_params={
                'eval_set': (X_val, y_val),
        }
    )
    hyperparam_optimizer.fit(X_train, y_train)

    return hyperparam_optimizer.best_estimator_

In [ ]:
# accuracy score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='accuracy')
_ = calculate_scores(best_estimator, X_val, y_val)

In [ ]:
# roc auc score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='roc_auc')
_ = calculate_scores(best_estimator, X_val, y_val)

In [ ]:
# f1 score
best_estimator = perform_bayes_search(clf, X_train, X_val, y_train, y_val, param_grid, scoring='f1')
_ = calculate_scores(best_estimator, X_val, y_val)